In [1]:
library(tidyr)
library(car)
library(emmeans)

# increase the number of characters that can be printed
options("width"=200)

Loading required package: carData



In [2]:
df <- read.csv("data/data.csv")

# only consider the cases with one face
df  <- subset(df, df$has_faces == 1)

# Face-ism in the person queries


In [3]:
# filter by the query type person, i.e. "person" and "intelligent person" queries
dfp <- subset(df, df$query_type =="person")

###create gender variable based on the face
dfp$gender <- NA
dfp[which(dfp$has_faces_male==1),"gender"]<-'Male'
dfp[which(dfp$has_faces_female==1),"gender"]<-'Female'

## 1. Anova table

In [4]:
# fit the model
fit <- lm(faceism ~ gender*query*engine+region+browser+wave, data=dfp)

# display general anova table (factors and interactions only)
print(Anova(fit, type="III"))

Anova Table (Type III tests)

Response: faceism
                     Sum Sq   Df  F value    Pr(>F)    
(Intercept)          15.062    1 399.1859 < 2.2e-16 ***
gender                0.138    1   3.6563   0.05590 .  
query                 2.028    1  53.7364 2.600e-13 ***
engine                0.486    3   4.2923   0.00494 ** 
region                0.031    2   0.4098   0.66377    
browser               0.003    1   0.0794   0.77813    
wave                  0.010    1   0.2539   0.61435    
gender:query          1.322    1  35.0238 3.439e-09 ***
gender:engine         3.562    3  31.4651 < 2.2e-16 ***
query:engine          3.275    3  28.9347 < 2.2e-16 ***
gender:query:engine   2.606    3  23.0262 8.163e-15 ***
Residuals           223.521 5924                       
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## 2. Contrast tests

### 2.1 Accross search engines

In [5]:
# full table of contrasts by search engine including. The interactions include those across 
# queries, but we only report the ones within the query term (i.e. "person Female / person Male" 
# and intelligent person Female / person Male")
pairs_engine <- pairs(emmeans(fit, ~ query * gender | engine),type = "response")

# print the pairs
print(pairs_engine)

engine = Baidu:
 contrast                                            estimate      SE   df t.ratio p.value
 intelligent person Female - person Female            0.18191 0.02482 5924   7.331  <.0001
 intelligent person Female - intelligent person Male  0.05324 0.02784 5924   1.912  0.2229
 intelligent person Female - person Male              0.00935 0.03005 5924   0.311  0.9896
 person Female - intelligent person Male             -0.12866 0.02351 5924  -5.474  <.0001
 person Female - person Male                         -0.17256 0.02608 5924  -6.616  <.0001
 intelligent person Male - person Male               -0.04390 0.02898 5924  -1.515  0.4287

engine = Bing:
 contrast                                            estimate      SE   df t.ratio p.value
 intelligent person Female - person Female           -0.05405 0.01495 5924  -3.615  0.0017
 intelligent person Female - intelligent person Male -0.04299 0.01356 5924  -3.170  0.0083
 intelligent person Female - person Male             -0.08

### 2.2 Accross interactions

In [6]:
# full table of contrasts by interactions of search engine and query
# we only report the ones within the query term (i.e. "person Female / person Mal" and
# intelligent person Female / person Male")
pairs_interaction <- pairs(emmeans(fit, ~ query * gender  | engine,type = "response"), interaction = "pairwise")

# print the pairs
print(pairs_interaction)

engine = Baidu:
 query_pairwise              gender_pairwise estimate     SE   df t.ratio p.value
 intelligent person - person Female - Male     0.2258 0.0382 5924   5.918  <.0001

engine = Bing:
 query_pairwise              gender_pairwise estimate     SE   df t.ratio p.value
 intelligent person - person Female - Male    -0.0159 0.0179 5924  -0.889  0.3741

engine = Google:
 query_pairwise              gender_pairwise estimate     SE   df t.ratio p.value
 intelligent person - person Female - Male    -0.1383 0.0235 5924  -5.894  <.0001

engine = Yandex:
 query_pairwise              gender_pairwise estimate     SE   df t.ratio p.value
 intelligent person - person Female - Male    -0.0654 0.0224 5924  -2.918  0.0035

Results are averaged over the levels of: region, browser, wave 


## 3 Confidence intervals

### 3.1 Accross search engines

In [7]:
# the confidence intervals here correspond to the ones in section 2.1
confint(pairs_engine)[,c(1,2,6,7)]

,contrast,engine,lower.CL,upper.CL
,<fct>,<fct>,<dbl>,<dbl>
1,intelligent person Female - person Female,Baidu,0.118138202,0.245675629
2,intelligent person Female - intelligent person Male,Baidu,-0.018310998,0.124798102
3,intelligent person Female - person Male,Baidu,-0.067871867,0.086568590
4,person Female - intelligent person Male,Baidu,-0.189067520,-0.068259207
5,person Female - person Male,Baidu,-0.239584201,-0.105532906
6,intelligent person Male - person Male,Baidu,-0.118367386,0.030577005
7,intelligent person Female - person Female,Bing,-0.092479273,-0.015626097
8,intelligent person Female - intelligent person Male,Bing,-0.077834552,-0.008138150
9,intelligent person Female - person Male,Bing,-0.115300248,-0.047041857


### 3.2 Accross interactions

In [8]:
# the confidence intervals here correspond to the ones in section 2.2
confint(pairs_interaction)[,c(1,2,3,7,8)]

,query_pairwise,gender_pairwise,engine,lower.CL,upper.CL
,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,intelligent person - person,Female - Male,Baidu,0.15100529,0.30059892
2,intelligent person - person,Female - Male,Bing,-0.05086397,0.01912800
3,intelligent person - person,Female - Male,Google,-0.18424630,-0.09227994
4,intelligent person - person,Female - Male,Yandex,-0.10940997,-0.02146886


## 2.3 Effect sizes

### 3.1 Accross search engines

In [9]:
# the effect sizess here correspond to the ones in section 2.1
summary(eff_size(pairs_engine, sigma = sigma(fit), edf = df.residual(fit), method = "identity"))[,c(1,2,3,6,7)]

,contrast,engine,effect.size,lower.CL,upper.CL
,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,(intelligent person Female - person Female),Baidu,0.93647699,0.685472036,1.18748195
2,(intelligent person Female - intelligent person Male),Baidu,0.27410372,-0.006954482,0.55516191
3,(intelligent person Female - person Male),Baidu,0.04812640,-0.255140387,0.35139319
4,(person Female - intelligent person Male),Baidu,-0.66237328,-0.899897145,-0.42484941
5,(person Female - person Male),Baidu,-0.88835059,-1.152064928,-0.62463625
6,(intelligent person Male - person Male),Baidu,-0.22597731,-0.518478880,0.06652426
7,(intelligent person Female - person Female),Bing,-0.27826922,-0.429264433,-0.12727401
8,(intelligent person Female - intelligent person Male),Bing,-0.22129851,-0.358215214,-0.08438180
9,(intelligent person Female - person Male),Bing,-0.41787759,-0.552123660,-0.28363151


### 3.2 Accross interactions

In [10]:
summary( eff_size(pairs_interaction, sigma(fit), df.residual(fit), method = "identity"))[,c(1,2,3,6,7)]


,contrast,engine,effect.size,lower.CL,upper.CL
,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,(intelligent person - person Female - Male),Baidu,1.16245430,0.7768233,1.54808533
2,(intelligent person - person Female - Male),Bing,-0.08169014,-0.2618594,0.09847908
3,(intelligent person - person Female - Male),Google,-0.71179389,-0.9488673,-0.47472051
4,(intelligent person - person Female - Male),Yandex,-0.33688936,-0.5633360,-0.11044277
